<a href="https://colab.research.google.com/github/techquila192/Gamma-Hadron-Classification/blob/main/Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [137]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score

In [138]:
cols=['fLength', 'fWidth', 'fSize', 'fConC', 'fConc1', 'fAsym', 'fM3Long', 'fM3Trans', 'fAlpha', 'fDist', 'class']
df = pd.read_csv("/content/sample_data/magic04.data",names=cols)
df_train,df_test,df_valid = np.split(df.sample(frac=1),[int(0.6*len(df)),int(0.8*len(df))])
df_train.head(10)

,fLength,fWidth,fSize,fConC,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
13712,105.1794,46.2561,3.5711,0.0841,0.0406,72.4321,61.0396,-33.1420,26.1867,225.4863,h
7705,27.8728,14.7857,2.3598,0.3843,0.2031,-3.2826,-22.8391,9.9961,2.2750,179.0940,g
4924,41.7353,15.5419,2.7435,0.3538,0.1940,-1.4304,28.6890,3.4026,11.4110,225.8450,g
18916,49.3296,23.3190,2.9009,0.1985,0.1137,12.0481,-32.4222,13.7545,14.0480,69.9318,h
6163,39.0781,23.4318,3.0726,0.2183,0.1096,4.7344,-15.2045,-14.3897,3.3991,171.9330,g
15225,60.1109,9.0810,2.8645,0.3374,0.1769,51.1900,11.9985,-4.8898,74.0560,130.7370,h
4452,19.4363,9.6894,2.4983,0.5937,0.3635,22.1912,12.1362,-3.5354,35.8010,154.5730,g
2542,96.4553,33.8560,4.0044,0.1104,0.0553,71.2174,78.5425,21.8686,1.7133,281.1890,g
3252,106.0080,20.3299,2.5384,0.3936,0.2012,-131.4040,82.9771,-12.0510,6.8644,196.7610,g
3521,34.8702,18.7861,2.6390,0.3238,0.1757,-25.4291,12.7094,-11.6604,25.1390,171.1630,g


# Data Scaling and Resampling

In [139]:
def scale_data(df):
    df["class"]=(df["class"]=="g").astype(int)
    df=df.fillna(df.median())
    scaler=StandardScaler()
    x_scale=df[cols[:-1]].values
    y=df[cols[-1]].values
    x_scaled=scaler.fit_transform(x_scale)
    x=x_scaled
    ros=RandomOverSampler()
    x,y=ros.fit_resample(x,y)
    data=np.hstack((x,np.reshape(y,(-1,1))))
    return x,y,data

In [140]:
x_train,y_train,train=scale_data(df_train)
x_test,y_test,test=scale_data(df_test)
x_valid, y_valid, valid=scale_data(df_valid)

# Training using Support Vector Machine(SVM)

In [141]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
model=SVC()
SVM_model=model.fit(x_train,y_train)

In [142]:
y_pred=SVM_model.predict(x_valid)
print(accuracy_score(y_valid,y_pred))

0.842845495311863


# Using KNN
Evaluating using k values from 1 to 20, it was found that the highest accuracy is achieved using a k value of 17

In [143]:
from sklearn.neighbors import KNeighborsClassifier
model=KNeighborsClassifier(n_neighbors=17)
KNN_model=model.fit(x_train,y_train)
y_pred=KNN_model.predict(x_valid)
print(accuracy_score(y_valid,y_pred))


0.8118630248675092


# Final Model
Better results are achieved using SVM

In [144]:
print(accuracy_score(y_test,SVM_model.predict(x_test)))

0.850996015936255
